In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [178]:
import cv2
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm

In [179]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.features = models.resnet18(weights='DEFAULT')
        # Modify the first layer to accept 1 channel input (for grayscale spectrograms)
        self.features.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        # Modify the final layer to output desired feature size
        self.features.fc = nn.Linear(self.features.fc.in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.features(x)
        x = self.softmax(x)
        return x

In [180]:
def extract_average_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    interval_frames = int(fps) # Frames to skip to get 1 frame per second
    frames = []
    for i in range(0, frame_count, interval_frames):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
    cap.release()
    # Calculate average frame
    averaged_frame = sum(frames) / len(frames)
    return averaged_frame

In [181]:
# def preprocess_image(averaged_frame):
#     # Convert the NumPy array to a PIL Image
#     averaged_frame_pil = Image.fromarray(averaged_frame.astype('uint8'))

#     # Resize and normalize the averaged frame
#     transform = transforms.Compose([
#         transforms.Resize((224, 224)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
#     ])
#     img_tensor = transform(averaged_frame_pil)
#     return img_tensor

def preprocess_image(averaged_frame):
    # Convert the NumPy array to a PIL Image
    averaged_frame_pil = Image.fromarray(averaged_frame.astype('uint8'))

    # Convert the image to grayscale
    averaged_frame_pil = averaged_frame_pil.convert('L')

    # Resize and normalize the averaged frame
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485], std=[0.229]),  # For grayscale images, only 1 channel
    ])
    img_tensor = transform(averaged_frame_pil)
    return img_tensor


In [182]:
def load_dataset(input_folder):
    X = []
    y = []
    # List all video files in the input folder
    video_files = [file for file in os.listdir(input_folder) if file.endswith(".flv")]
    for video_file in video_files:
        video_path = os.path.join(input_folder, video_file)
        averaged_frame = extract_average_frames(video_path)
        img_tensor = preprocess_image(averaged_frame)
        X.append(img_tensor)
        # Extract label from video filename (assuming filename is in format "label_videoID.mp4")
        label = video_file.split("_")[2]
        if label == "HAP":
            y.append(0)
        elif label == "SAD":
            y.append(1)
        elif label == "ANG":
            y.append(2)
        # print(video_path, label)
    return X, y

In [183]:
def train_model(model, criterion, optimizer, train_loader, device):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [184]:
def test_model(model, criterion, test_loader, device):
    model.eval()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct_preds += (predicted == labels).sum().item()
            total_preds += labels.size(0)
    epoch_loss = running_loss / len(test_loader.dataset)
    accuracy = correct_preds / total_preds
    return epoch_loss, accuracy

In [185]:
# def extract_features_from_folder(input_folder):
#     # Initialize the model
#     model = CNN(num_classes=3)  # 3 classes for HAPPY, SAD, ANGRY
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.eval()  # Set the model to evaluation mode

#     # List all files in the input folder
#     files = os.listdir(input_folder)

#     # Iterate over files in the folder
#     for filename in files:
#         if filename.endswith(".png"):  # Assuming mel spectrograms are stored as PNG files
#             input_path = os.path.join(input_folder, filename)
#             img_tensor = preprocess_image(input_path)
#             img_tensor = img_tensor.to(device)
#             with torch.no_grad():
#                 output_features = model(img_tensor)
#             print(f"Features extracted for {filename}: {output_features}")


In [186]:
# extract_features_from_folder('/content/drive/MyDrive/csci535/melspec')

In [187]:
# !python3 melspec_to_features_cnn.py /content/drive/MyDrive/csci535/melspec


In [188]:
if __name__ == "__main__":
    # Check if input arguments are provided
    # if len(sys.argv) != 2:
    #     print("Usage: python video_to_features_cnn.py input_folder")
    #     sys.exit(1)

    # input_folder = sys.argv[1]
    input_folder = '/content/drive/MyDrive/csci535/videos'
    # Check if input folder exists
    if not os.path.exists(input_folder):
        print("Input folder does not exist.")
        sys.exit(1)

    # Load dataset and split into train and test sets
    X, y = load_dataset(input_folder)
    print(f"Total number of samples: {len(X)}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    print(f"Number of train samples: {len(X_train)}", f"Number of test samples: {len(X_test)}")
    # Initialize the model
    model = CNN(num_classes=3)  # 3 classes for HAPPY, SAD, ANGRY
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Define loss function and optimizer
    _lr = 0.001
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=_lr)

    # Create data loaders
    _bs = 32
    train_loader = torch.utils.data.DataLoader(list(zip(X_train, y_train)), batch_size=_bs, shuffle=True)
    test_loader = torch.utils.data.DataLoader(list(zip(X_test, y_test)), batch_size=_bs)
    print(f"Batch size: {_bs}", f"lr: {_lr}")
    # Training loop
    num_epochs = 50
    for epoch in range(num_epochs):
        print("Epoch " + str(epoch))
        train_loss, train_accuracy = train_model(model, criterion, optimizer, train_loader, device)
        test_loss, test_accuracy = test_model(model, criterion, test_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Total number of samples: 273
Number of train samples: 191 Number of test samples: 82
Batch size: 32 lr: 0.001
Epoch 0


100%|██████████| 3/3 [00:00<00:00, 41.11it/s]


Epoch 1/50, Train Loss: 1.1442, Train Accuracy: 0.3298, Test Loss: 1.1696, Test Accuracy: 0.3293
Epoch 1


100%|██████████| 3/3 [00:00<00:00, 41.34it/s]


Epoch 2/50, Train Loss: 1.1817, Train Accuracy: 0.3455, Test Loss: 1.2308, Test Accuracy: 0.3171
Epoch 2


...

Epoch 48/50, Train Loss: 0.7159, Train Accuracy: 0.8377, Test Loss: 1.0886, Test Accuracy: 0.4390
Epoch 48


100%|██████████| 3/3 [00:00<00:00, 41.31it/s]


Epoch 49/50, Train Loss: 0.6767, Train Accuracy: 0.8743, Test Loss: 1.0828, Test Accuracy: 0.4634
Epoch 49


100%|██████████| 3/3 [00:00<00:00, 40.43it/s]

Epoch 50/50, Train Loss: 0.6535, Train Accuracy: 0.9058, Test Loss: 1.0802, Test Accuracy: 0.4390


In [189]:
torch.save(model.state_dict(), 'ResNet18_video_'+str(num_epochs)+'_'+str(_bs)+'_'+str(_lr))

In [190]:
# ! ls -lh /content/

In [191]:
!cp '/content/ResNet18_video_50_32_0.001' '/content/drive/MyDrive/csci535/models'